## **1. Đọc dữ liệu**

In [31]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics

In [33]:
path_train = "/content/drive/MyDrive/ids_ga_ml/UNSW_NB15_training-set.csv"
path_test = "/content/drive/MyDrive/ids_ga_ml/UNSW_NB15_testing-set.csv"

In [34]:
df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)

In [35]:
df_train.drop(['label'], axis=1, inplace=True)
df_test.drop(['label'], axis=1, inplace=True)

In [36]:
# prompt: tạo ra df mới từ việc concat df_train với df_test

df = pd.concat([df_train, df_test])


In [37]:
y = df['attack_cat']
X = df.drop('attack_cat', axis=1, inplace=False)

In [38]:
# y_test = df_test['attack_cat']
# X_test = df_test.drop('attack_cat', axis=1)




In [39]:
# prompt: use train test slip saprarte X_train, y_train, X_test, y_test from X,y

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## **2. Xử lý dữ liệu**

In [40]:
# # prompt: import thư viện, sinh mẫu để cân bằng dữ liệu theo cột attack_cat

# from imblearn.over_sampling import RandomOverSampler

# # Tách dữ liệu và nhãn
# X_train = df_train.drop('attack_cat', axis=1)
# y_train = df_train['attack_cat']

# # Sinh mẫu để cân bằng dữ liệu theo cột attack_cat
# ros = RandomOverSampler(sampling_strategy='all')
# X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# # Hiển thị số lượng mẫu của mỗi lớp sau khi cân bằng dữ liệu
# print(y_train_resampled.value_counts())


In [41]:
def min_max_scaler(data):
  scaled_dataset = (data-data.min())/(data.max()-data.min())
  return scaled_dataset

In [42]:
# prompt: tạo 1 function, đầu vào là 1 dataframe, tạo 1 biến tên cate-col để lưu trữ list là tên các cột có kiểu dữ liệu object, ròi dùng lable encode xử lý các cột đấy
from sklearn.preprocessing import LabelEncoder

# Creating a instance of label Encoder.
le = LabelEncoder()

def tranformdata(df):
  for col in df.columns:
    if df[col].dtype == 'object':

      df[col] = le.fit_transform(df[col].astype(str))
  return df

In [43]:
# y_train = y_train_resampled
# X_train = X_train_resampled

In [44]:
# y_train = df_train['attack_cat']
# df_train.drop(['attack_cat', 'label'], axis=1, inplace=True)

In [45]:
# y_test = df_test['attack_cat']
# df_test.drop(['attack_cat', 'label'], axis=1, inplace=True)

In [46]:
y_train.info()

<class 'pandas.core.series.Series'>
Index: 206138 entries, 9459 to 121958
Series name: attack_cat
Non-Null Count   Dtype 
--------------   ----- 
206138 non-null  object
dtypes: object(1)
memory usage: 3.1+ MB


In [47]:
X_train = tranformdata(X_train)
X_test = tranformdata(X_test)
X_train = min_max_scaler(X_train)
X_test = min_max_scaler(X_test)
# y_train = tranformdata(y_train)
# y_test = tranformdata(y_test)

In [48]:
# prompt: viết một hàm  mapping các giá trị trong attack cat sang thành int

def map_attack_cat(attack_cat):


  mapping = {
    'Normal': 0,
    'Generic': 1,
    'Exploits': 2,
    'Fuzzers': 3,
    'DoS': 4,
    'Reconnaissance': 5,
    'Analysis': 6,
    'Backdoor': 7,
    'Shellcode': 8,
    'Worms': 9,
  }

  return mapping.get(attack_cat, None)

y_train = y_train.apply(map_attack_cat)
y_test = y_test.apply(map_attack_cat)



In [49]:
y_train.unique()

array([2, 1, 0, 3, 4, 5, 6, 8, 7, 9])

## **4. Model KNN, XGBOOST, SVM**


### **4.2 Tạo model**

In [57]:
# prompt: import thư viện, định nghĩa ra các function với biến đầu vào là X,y , function là các model:svm,knn,xgboost , đầu ra là in các chỉ số accracy và recall
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score
from sklearn.preprocessing import LabelEncoder



def svm(X_train, X_test, y_train, y_test):
  # Split the data into training and test sets
  # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Create a SVM model
  model = SVC(kernel='linear', C=1, gamma='scale', class_weight='balanced')

  # Train the model
  model.fit(X_train, y_train)

  # Predict on the test set
  y_pred = model.predict(X_test)

  # Calculate accuracy and recall
  accuracy = accuracy_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred, average='weighted')

  # Return the scores
  return accuracy, recall


def knn(X_train, X_test, y_train, y_test):

  # Create a KNN model
  model = KNeighborsClassifier(n_neighbors=20)

  # Train the model
  model.fit(X_train, y_train)

  # Predict on the test set
  y_pred = model.predict(X_test)

  # Calculate accuracy and recall
  accuracy = accuracy_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred, average='weighted')

  # Return the scores
  return accuracy, recall


def xgboost(X_train, X_test, y_train, y_test):
  # le = LabelEncoder()
  # y_train = le.fit_transform(y_train)
  # y_test = le.fit_transform(y_test)
  # Split the data into training and test sets
  # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

  # Create a XGBoost model
  model = XGBClassifier( n_estimators=100, max_depth=100 )

  # Train the model
  model.fit(X_train, y_train)

  # Predict on the test set
  y_pred = model.predict(X_test)

  # Calculate accuracy and recall
  accuracy = accuracy_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred, average='weighted')

  # Return the scores
  return accuracy, recall



## **5. Đánh giá**

In [51]:
xg_acc, xg_recall = xgboost(X_train,X_test,y_train,y_test)
print("xgboost accuracy: ", xg_acc)
print("xgboost recall: ", xg_recall)

xgboost accuracy:  0.7804404773454934
xgboost recall:  0.7804404773454934


In [52]:
knn_acc, knn_recall = knn(X_train,X_test,y_train,y_test)
print("knn accuracy: ", knn_acc)
print("knn recall: ", knn_recall)

knn accuracy:  0.7926845832929077
knn recall:  0.7926845832929077


In [ ]:
svm_acc, svm_recall = svm(X_train,X_test,y_train,y_test)
print("svm accuracy: ", svm_acc)
print("svm recall: ", svm_recall)

svm accuracy:  0.7882992141263219
svm recall:  0.7882992141263219
